In [1]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

  Activating project at `~/Projects/DIVAnd-FAIR-EASE`


In [2]:
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, footprintdir])

2-element Vector{String}:
 "../data"
 "../Footprint"

In [3]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [11]:
datasource = "Euro-Argo"
domain = [12., 18., 43., 46.]
parameter1 = "TEMP"
if datasource == "World Ocean Database"
    parameter1 = "Temperature"
elseif datasource == "EMODnet Chemistry"  
    parameter1 = "ITS_90_water_temperature"
elseif datasource == "SeaDataNet CDI TS"
    parameter1 = "ITS-90 water temperature"
end
parameter2 = "PSAL"
datestart = Dates.Date(1950, 1, 1)
dateend = Dates.Date(2024, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 10.
regionname = "NorthAdriaticSea"
@info("$(datestart) → $(dateend), $(parameter1)")

[ Info: 1950-01-01 → 2024-01-01, TEMP


In [7]:
beaconURL = beacon_services[datasource]
footprintURL = joinpath(beaconURL, "api/datasets/footprint")
datasource_name = replace(datasource, " " => "_")
footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
@info("Footprint endpoint: $(footprintURL)")
@info("Footprint file: $(footprintfile)")

[ Info: Footprint endpoint: https://beacon-argo.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_Euro-Argo.json


In [8]:
if isfile(footprintfile)
    @info("Footprint file already downloaded")
else
    @info("Writing Footprint file")
    open(footprintfile, "w") do io
        r = HTTP.request("GET", footprintURL, 
            ["Authorization" => "Bearer $(token)"],
            response_stream=io)
        @info(r.status)
    end
end

[ Info: Writing Footprint file
[ Info: 200


In [9]:
# Note 1: the JSON file is 193M, just to get the list of variables?

In [10]:
data = JSON.parsefile(footprintfile);
attributes = data["unique_column_attributes"]
params = sort(collect(keys(attributes)));
parameter1 in params
@show(params);

params = ["BBP470", "BBP470_ADJUSTED", "BBP470_ADJUSTED_ERROR", "BBP470_ADJUSTED_QC", "BBP470_QC", "BBP470_dPRES", "BBP532", "BBP532_ADJUSTED", "BBP532_ADJUSTED_ERROR", "BBP532_ADJUSTED_QC", "BBP532_QC", "BBP532_dPRES", "BBP700", "BBP700_2", "BBP700_2_ADJUSTED", "BBP700_2_ADJUSTED_ERROR", "BBP700_2_ADJUSTED_QC", "BBP700_2_QC", "BBP700_2_dPRES", "BBP700_ADJUSTED", "BBP700_ADJUSTED_ERROR", "BBP700_ADJUSTED_QC", "BBP700_QC", "BBP700_dPRES", "BETA_BACKSCATTERING470", "BETA_BACKSCATTERING470_QC", "BETA_BACKSCATTERING532", "BETA_BACKSCATTERING532_MED", "BETA_BACKSCATTERING532_MED_QC", "BETA_BACKSCATTERING532_QC", "BETA_BACKSCATTERING532_STD", "BETA_BACKSCATTERING532_STD_QC", "BETA_BACKSCATTERING700", "BETA_BACKSCATTERING700_2", "BETA_BACKSCATTERING700_2_QC", "BETA_BACKSCATTERING700_MED", "BETA_BACKSCATTERING700_MED_QC", "BETA_BACKSCATTERING700_QC", "BETA_BACKSCATTERING700_STD", "BETA_BACKSCATTERING700_STD_QC", "BISULFIDE", "BISULFIDE_ADJUSTED", "BISULFIDE_ADJUSTED_ERROR", "BISULFIDE_ADJUSTED

631-element Vector{String}:
 "BBP470"
 "BBP470_ADJUSTED"
 "BBP470_ADJUSTED_ERROR"
 "BBP470_ADJUSTED_QC"
 "BBP470_QC"
 "BBP470_dPRES"
 "BBP532"
 "BBP532_ADJUSTED"
 "BBP532_ADJUSTED_ERROR"
 "BBP532_ADJUSTED_QC"
 "BBP532_QC"
 "BBP532_dPRES"
 "BBP700"
 ⋮
 "UV_INTENSITY_NITRATE"
 "UV_INTENSITY_NITRATE_QC"
 "VERTICAL_SAMPLING_SCHEME"
 "VK_PH"
 "VK_PH_QC"
 "VRS_PH"
 "VRS_PH_MED"
 "VRS_PH_MED_QC"
 "VRS_PH_QC"
 "VRS_PH_STD"
 "VRS_PH_STD_QC"
 "WMO_INST_TYPE"

In [16]:
query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
    mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

"{\"query_parameters\":[{\"column_name\":\"TEMP\",\"alias\":\"TEMP\",\"optional\":false},{\"column_name\":\"PSAL\",\"alias\":\"PSAL\",\"optional\":true},{\"column_name\":\"JULD\",\"alias\":\"TIME\"},{\"column_name\":\"PRES\",\"alias\":\"DEPTH\"},{\"column_name\":\"LONGITUDE\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"" ⋯ 20 bytes ⋯ "ATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":0,\"max\":27028},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":10.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0}],\"output\":{\"format\":\"netcdf\"}}"

In [157]:
# Note 2: cannot work with such a long list of parameters!!

In [14]:
filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
filename

"../data/NorthAdriaticSea_Euro-Argo_TEMP_19500101-20240101_0-10m.nc"

In [17]:
@info("Data will be written in file:\n$(filename)")

if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-type"=> "application/json",
         "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end

@info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")

┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_Euro-Argo_TEMP_19500101-20240101_0-10m.nc
[ Info: File already downloaded
[ Info: 200


  1.086903 seconds (78.35 k allocations: 5.869 MiB, 9.19% compilation time)


[ Info: NetCDF file size: 0.3M


In [44]:
query

"{\"query_parameters\":[{\"column_name\":\"TEMP\",\"alias\":\"TEMP\"},{\"column_name\":\"JULD\",\"alias\":\"TIME\"},{\"column_name\":\"DEPH\",\"alias\":\"DEPTH\"},{\"column_name\":\"LONGITUDE\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"LATITUDE\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":21915,\"max\":22645},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":10.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0},{\"for_query_parameter\":\"TEMP\",\"min\":-2.0,\"max\":30.0}],\"output\":{\"format\":\"netcdf\"}}"